In [ ]:
import time, random
from selenium import webdriver
from db_conn import select_data, push_prod_data2

def get_ext2(detail_price, ext1):
    if 'kg' in ext1:
        hund_price = float(detail_price.replace(',', ''))
        weight = float(ext1.replace('kg', ''))
        weight = weight * 1000
        return str(int(hund_price / weight * 100))
    elif 'g' in ext1:
        hund_price = float(detail_price.replace(',', ''))
        weight = float(ext1.replace('g', ''))
        return str(int(hund_price / weight * 100))
    return ''

driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(2)

In [ ]:
page = 0
pagelimit = 100

temp_list = select_data({'page': page, 'pagelimit': pagelimit})
for temp in temp_list:
    parent_pcode = temp[1]
    driver.get(f'https://search.shopping.naver.com/catalog/{parent_pcode}')
    time.sleep(random.randint(1, 2))
    style_detail = driver.find_element_by_css_selector('div[class^="style_detail"]')
    ext_list = style_detail.find_elements_by_css_selector('div[class^="filter_condition_group"] > ul > li')
    if len(ext_list) == 0:
        pcode = parent_pcode+'-1'
        ext = driver.find_element_by_css_selector('div[class^="productList_product"] > a').text
        detail_price = driver.find_element_by_css_selector('div[class^="productList_price"] > a > span > em').text
        detail_price = detail_price.replace('원', '')
        ext1 = ext.split(' ')[-1]
        ext2 = get_ext2(detail_price, ext1)
        push_prod_data2({
            'parent_pcode': parent_pcode,
            'pcode': pcode,
            'price': detail_price,
            'href': 'check',
            'ext1': ext1,
            'ext2': ext2+'원/100g',
        })

    for i, ext in enumerate(ext_list):
        pcode = parent_pcode+'-'+str(i+1)
        detail_price = ext.find_element_by_css_selector('span[class^="filter_price"]').text
        detail_price = detail_price.replace('모바일', '')
        detail_price = detail_price.replace('원', '')
        ext1 = ext.find_element_by_css_selector('span[class^="filter_text"]').text
        ext2 = get_ext2(detail_price, ext1)
        push_prod_data2({
            'parent_pcode': parent_pcode,
            'pcode': pcode,
            'price': detail_price,
            'href': '',
            'ext1': ext1,
            'ext2': ext2+'원/100g',
        })

    time.sleep(random.randint(1, 3))

In [ ]:
driver.quit()